In [3]:
#pip install openpyxl

In [4]:
#pip install xlrd

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  # <<-- CORRECCIÓN: IMPORTAR PLT
import seaborn as sns          # <<-- CORRECCIÓN: IMPORTAR SNS
import os

df = pd.read_excel("Shark.xls")


In [6]:
df.columns

Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatal Y/N', 'Time', 'Species ',
       'Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1',
       'original order', 'Unnamed: 21', 'Unnamed: 22'],
      dtype='object')

In [7]:
df.drop(columns = ['pdf', 'href formula', 'href', 'Case Number', 'Case Number.1',
    'original order', 'Unnamed: 21', 'Unnamed: 22'],axis=1,inplace=True)


In [8]:
df.columns

Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatal Y/N', 'Time', 'Species ',
       'Source'],
      dtype='object')

In [9]:
print(df.columns)

# Check missing values
print(df.isnull().sum())

Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatal Y/N', 'Time', 'Species ',
       'Source'],
      dtype='object')
Date            0
Year            2
Type           18
Country        50
State         486
Location      567
Activity      585
Name          219
Sex           579
Age          2994
Injury         35
Fatal Y/N     561
Time         3526
Species      3131
Source         20
dtype: int64


In [10]:
for col in ['Country', 'State', 'Location']:
    df[col] = df[col].astype(str).str.strip().str.title()

# Replacing 'nan' strings with actual NaN
df.replace('Nan', pd.NA, inplace=True)

# filling missing with 'Unknown'
df[['Country', 'State', 'Location']] = df[['Country', 'State', 'Location']].fillna('Unknown')

In [11]:
df[['Country', 'State', 'Location']].head(10)

,Country,State,Location
0,Columbia,"Bolivar, Del Isolate",Catagena Province
1,Australia,Queensland,Cook Esplanade Thursday Island
2,Australia,South Australia,Kangaroo Island
3,Usa,Off California,Catalina Island
4,Costa Rica,Unknown,Cocos Islands
5,Australia,Nsw,Long Reef Sydney
6,Usa,Florida,Horseshoe Reef Key Largo
7,Usa,Texas,Galveston
8,Australia,Nsw,Cabarita Beach
9,Bahamas,Atlantic Ocean Near Big Grand Cay,North Of Grand Bahama Near Freeport


In [12]:
df['Activity'] = df['Activity'].astype(str).str.strip().str.title()

In [13]:
def categorize_activity(act):
    act = str(act).lower()
    if 'surf' in act:
        return 'Surfing'
    elif 'swim' in act:
        return 'Swimming'
    elif 'wade' in act:
        return 'Wading'
    elif 'snorkel' in act:
        return 'Snorkeling'
    elif 'dive' in act:
        return 'Diving'
    elif 'fish' in act:
        return 'Fishing'
    elif act == 'Nan' or act == '':
        return 'Unknown'
    else:
        return 'Other'

In [14]:
df['Activity_Category'] = df['Activity'].apply(categorize_activity)

In [21]:
df['Activity_Category'].unique()

array(['Swimming', 'Surfing', 'Other', 'Snorkeling', 'Fishing', 'Wading',
       'Diving', 'Unknown'], dtype=object)

In [15]:
df[['Activity', 'Activity_Category']].head(10)

,Activity,Activity_Category
0,Swimming With Sharks,Swimming
1,Fishing/Swimming,Swimming
2,Surfing,Surfing
3,Swimming,Swimming
4,Diving-Tagging Sharks,Other
5,Surfing,Surfing
6,Snorkeling,Snorkeling
7,Swimming,Swimming
8,Surfing,Surfing
9,Spearfishing,Fishing


In [16]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Extract Month
df['Month'] = df['Date'].dt.month

# Extract Season
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Unknown'

df['Season'] = df['Month'].apply(get_season)

/var/folders/lr/d46q9z0s2zndy_5npx2_zwq00000gn/T/ipykernel_29247/2476793777.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


In [17]:
# to string, quitar espacios y poner en minúsculas
df['Sex'] = df['Sex'].astype(str).str.strip().str.lower()


df['Sex'] = df['Sex'].replace({
    'm': 'Male',
    'male': 'Male',
    'f': 'Female',
    'female': 'Female'
})

# Reemplazar strings 'nan' con valor NaN real
df['Sex'] = df['Sex'].replace('nan', pd.NA)

# Rellenar faltantes con 'Unknown'
df['Sex'] = df['Sex'].fillna('Unknown')

In [18]:
# checking
df['Sex'].head(10)

0      Male
1      Male
2      Male
3      Male
4      Male
5      Male
6      Male
7    Female
8      Male
9      Male
Name: Sex, dtype: object

In [28]:
df['Sex'].unique()

array(['Male', 'Female', 'Unknown', 'lli', 'm x 2', 'n', '.'],
      dtype=object)

In [19]:
# --- 1. cleaning 'Age' ---

# Convertir a string, quitar espacios y poner en mayúsculas
df['Age'] = df['Age'].astype(str).str.strip().str.upper()

# Reemplazar posibles valores no numéricos y de texto con NaN (np.nan)
df['Age'] = df['Age'].replace({'ADULT': np.nan, 'X': np.nan, 'YOUNG': np.nan, 'TEEN': np.nan,
                               'NAN': np.nan, '': np.nan, '-': np.nan, '18 MONTHS': 1.5, '9 MONTHS': 0.75, '8 MONTHS': 0.67})

# Intentar convertir a numérico, forzando errores a NaN
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')

# Rellenar faltantes con 0 (como marcador de "Desconocido/Imputado")
df['Age'] = df['Age'].fillna(0)

# Finalmente, convertir a tipo entero
df['Age'] = df['Age'].astype(int)


In [20]:
# --- 3. FILTRADO POR HIPÓTESIS: USA (12-20 años) ---
# Nos enfocamos en ataques modernos (desde 1950) en USA, para el grupo de edad 12-20.
df_hypothesis = df[(df['Country'] == 'Usa') & 
                   (df['Age'] >= 12) & 
                   (df['Age'] <= 20) &
                   (df['Year'] >= 1950) 
                   ].copy()


# Top 5 Spots más peligrosos (State)
top_states = df_hypothesis['State'].value_counts().head(5)

# Top 5 Actividades para este grupo
top_activities_hypo = df_hypothesis['Activity_Category'].value_counts().drop(['Unknown', 'Other'], errors='ignore').head(5)


# --- 4. EDA Y VISUALIZACIÓN (DÍA 4) ---

# Top 5 States (Spots peligrosos)
plt.figure(figsize=(10, 6))

sns.barplot(x=top_states.index, y=top_states.values, palette='magma', hue=top_states.index, legend=False) 
plt.title('Ataques de Tiburón en USA por Estado (Edad 12-20 años, desde 1950)')
plt.xlabel('Estado (Spot)')
plt.ylabel('Número de Ataques')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('hypo_top_5_states_usa_12_20_final.png')
plt.close()

# Top 5 Actividades
plt.figure(figsize=(8, 5))

sns.barplot(x=top_activities_hypo.index, y=top_activities_hypo.values, palette='viridis', hue=top_activities_hypo.index, legend=False)
plt.title('Top 5 Actividades (USA, Edad 12-20 años, desde 1950)')
plt.xlabel('Actividad')
plt.ylabel('Número de Ataques')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('hypo_top_5_activities_usa_12_20_final.png')
plt.close()

# Imprimir las tablas de agregación
print("\n--- Final RESULTS (USA, 12-20 yo) ---")

print("\n--- Top 5 Spots MOST DANGEROUS (States) ---")
print(top_states.to_markdown())

print("\n--- Top 5 Activities ---")
print(top_activities_hypo.to_markdown())


--- Final RESULTS (USA, 12-20 yo) ---

--- Top 5 Spots MOST DANGEROUS (States) ---
| State          |   count |
|:---------------|--------:|
| Florida        |     349 |
| South Carolina |      35 |
| California     |      33 |
| North Carolina |      33 |
| Hawaii         |      30 |

--- Top 5 Activities ---
| Activity_Category   |   count |
|:--------------------|--------:|
| Surfing             |     253 |
| Swimming            |      81 |
| Fishing             |      26 |
| Snorkeling          |       5 |
| Wading              |       2 |
